# install Library

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
import torch
import pickle

# Data Preparation 

In [13]:
device = torch.device('cpu')

# Hyperparameters
train_proportion = 0.80
hidden_layer_size = 70
learning_rate = 0.0001
batch_size = 64
epochs = 20

In [14]:
# train_proportion = 0.8
# train_dataset = pd.read_csv("stock_data.csv")
# train_size = int(train_proportion * len(train_dataset))
# train_datasett.drop("id", axis = 1, inplace=True)

# ax = full_dataset.groupby('sentiment').count().plot(kind="bar", title="Distribution of data", legend=False)
# ax.set_xticklabes(['Negative','Positive'],rotation=0)

In [15]:
#Dataload
with open("encoded_dataframe", "rb") as encoded_dataframe:
    encoded_df = pickle.load(encoded_dataframe)

# Shuffle data and split into training and testing data
train_dataset = encoded_df.sample(frac = train_proportion)
test_dataset = encoded_df.drop(train_dataset.index)

train_size = train_dataset.shape[0]
test_size = test_dataset.shape[0]

train_tensor = torch.utils.data.TensorDataset(torch.stack(tuple(train_dataset[0])).type(torch.float32).to(device), torch.stack(tuple(train_dataset[1])).to(device))
test_tensor = torch.utils.data.TensorDataset(torch.stack(tuple(test_dataset[0])).type(torch.float32).to(device), torch.stack(tuple(test_dataset[1])).to(device))

train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_tensor, batch_size = 1)

In [16]:
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

# # Remove the text column because the model does not accept raw text as an input:
# # tokenized_datasets = tokenized_datasets.remove_columns(["text"])

# # Rename the label column to labels because the model expects the argument to be named labels:
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# # Set the format of the dataset to return PyTorch tensors instead of lists:
# tokenized_datasets.set_format("torch")



In [18]:
#Optimizer and learning rate scheduler
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
# Training loop
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


  0%|                                                                                          | 0/219 [00:06<?, ?it/s]


AttributeError: 'list' object has no attribute 'items'

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()